# **Games with AVG starting player valuations**

In [132]:
import pandas as pd
valuations = pd.read_csv("player_valuations.csv")
valuations["date"] = pd.to_datetime(valuations.date)
valuations["season"] = valuations.apply(lambda x: x.date.year, axis=1)
valuations = valuations.groupby(["player_id", "season"]).max("market_value_in_eur").reset_index()
valuations = valuations[["player_id", "market_value_in_eur", "season"]]

games = pd.read_csv("games.csv")
games = games[[
    "game_id", "season", "home_club_id", "away_club_id",
    "home_club_goals", "away_club_goals"
]]

game_lineups = pd.read_csv("game_lineups.csv")
game_lineups = game_lineups[game_lineups["type"] == "starting_lineup"]
game_lineups = game_lineups[["game_id", "club_id", "player_id"]]

In [133]:
df = games.merge(game_lineups, on="game_id").merge(valuations, on=["player_id", "season"])
market_values = df.groupby(["game_id", "club_id"])[["market_value_in_eur"]].mean().reset_index()

In [134]:
df = games.merge(market_values, left_on=["game_id", "home_club_id"],
                 right_on=["game_id", "club_id"], suffixes=["", "_"])
df = df[[
    "game_id", "season", "home_club_id", "away_club_id", "home_club_goals",
    "away_club_goals", "market_value_in_eur"
]]

df.columns = [
    "game_id", "season", "home_club_id", "away_club_id", "home_club_goals",
    "away_club_goals", "home_avg_market_value"
]

df = df.merge(market_values, left_on=["game_id", "away_club_id"],
              right_on=["game_id", "club_id"], suffixes=["", "_"])
df = df[[
    "game_id", "season", "home_club_id", "away_club_id", "home_club_goals",
    "away_club_goals", "home_avg_market_value", "market_value_in_eur"
]]

df.columns = [
    "game_id", "season", "home_club_id", "away_club_id", "home_club_goals",
    "away_club_goals", "home_avg_market_value", "away_avg_market_value"
]

In [135]:
def result(data):
  home=data["home_club_goals"]
  away=data["away_club_goals"]
  if home > away:
    val = 1
  elif home == away:
    val = 0
  elif home < away:
    val = -1
  return val

df["result"] = df.apply(result, axis=1)
df["goal_dif"] = df["home_club_goals"] - df["away_club_goals"]

In [136]:
df

,game_id,season,home_club_id,away_club_id,home_club_goals,away_club_goals,home_avg_market_value,away_avg_market_value,result,goal_dif
0,4087929,2023,350,873,0.0,1.0,3.722727e+06,1.965455e+07,-1,-1.0
1,4087952,2023,762,31,1.0,2.0,3.645455e+07,4.877273e+07,-1,-1.0
2,4087962,2023,281,931,5.0,1.0,6.536364e+07,1.190909e+07,1,4.0
3,4087965,2023,405,873,3.0,1.0,3.227273e+07,1.601818e+07,1,2.0
4,4089283,2023,678,443,1.0,0.0,6.272727e+05,4.100000e+05,1,1.0
...,...,...,...,...,...,...,...,...,...,...
1560,4135637,2023,122,383,1.0,3.0,2.000000e+06,9.281818e+06,-1,-2.0
1561,4097959,2023,383,467,4.0,0.0,9.936364e+06,1.236364e+06,1,4.0
1562,4013659,2023,512,3684,3.0,1.0,3.700000e+06,2.750000e+05,1,2.0
1563,4135635,2023,383,122,4.0,1.0,1.018182e+07,1.400000e+06,1,3.0
